In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# !pip install -q pycaret
# from pycaret.classification import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import *

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
submission

In [ ]:
label_encoder = LabelEncoder()
y_labels = label_encoder.fit_transform(train['NObeyesdad'])

In [ ]:
test_ids = test['id'].copy()
print(train.shape)
train = train.drop('NObeyesdad',axis=1)
print(train.shape)
train['dataset']=1
test['dataset']=0

In [ ]:
df = pd.concat([train,test]).drop('id',axis=1)
print(df.shape)
df = pd.get_dummies(df)*1
print(df.shape)

In [ ]:
cond = df['dataset']==1
train = df[cond].drop('dataset',axis=1).reset_index(drop=True)
test = df[~cond].drop('dataset',axis=1).reset_index(drop=True)
print(train.shape, test.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, y_labels, test_size=0.3, random_state=69, stratify=y_labels)
X_train.shape, y_train.shape,X_test.shape,y_test.shape

In [ ]:
%%time
voting_clf = VotingClassifier(estimators=[
    ('cat_clf',CatBoostClassifier()),
    ('xgb_clf',XGBClassifier()),
    ('gbc_clf',GradientBoostingClassifier()),
    ('rf_clf',RandomForestClassifier()),
    ('lgbm_clf',LGBMClassifier())
    ],
    voting='soft'
    )
voting_clf.fit(X_train,y_train)

In [ ]:
y_pred = voting_clf.predict(X_test)

In [ ]:
print("Evaluation Metrics (Voting Classifier)")
print(classification_report(y_test, y_pred))

In [ ]:
voting_clf.fit(train, y_labels)
y_pred = voting_clf.predict(test)

In [ ]:
submission_final2 = pd.DataFrame()

In [ ]:
submission_final2 = pd.concat([test_ids,pd.Series(y_pred,name='NObeyesdad')],axis=1)
submission_final2

In [ ]:
submission_final2['NObeyesdad'] = label_encoder.inverse_transform(submission_final2['NObeyesdad'])

In [ ]:
submission_final2

In [ ]:
submission_final2.to_csv('submission.csv',index=False,header=True)